1. LLM - Large Language Model 호출
2. Chat - model 사용
3. LLM과 chat model 의 차이점

- 세부사항에 있다. 예를 들어 사용 모델(text-davinci-003 / gpt-3.5-turbo),
- chat model 이 훨씬 싸다.

랭체인을 사용하면 llm 모델들에 접근할 수 있다. 쉽게


In [1]:
from langchain.llms import OpenAI, OpenAIChat
from langchain.chat_models import ChatOpenAI

# chat model: 대화에 특화된 모델
chat = ChatOpenAI()

# 대화에 특화되어 있어서 꼭 질문을 보내지 않아도 된다.
# 메세지를 보내보자.
chat.predict("what is python?")

'Python is a high-level, versatile, and interpreted programming language known for its simplicity and readability. It is widely used for web development, data analysis, artificial intelligence, scientific computing, and more. Python code is typically easy to write and understand, making it a popular choice for beginners and experienced programmers alike.'

llm 과 chat model 둘 다 텍스트를 predict 할 수 있다.  
chat model 은 대화에 최적화되어 있는데, 단지 질문을 받을 수 있을 뿐 아니라, 대화를 할 수 있다.

대화. 즉, 여러 메세지 묶음이라는 의미.  
상대로서 대화의 맥락에 맞게 대답할 수 있다는 뜻이다.


In [ ]:
# model 의 설정 바꾸기 - 모델의 constructure 에서 할 수 있다.
chat = ChatOpenAI(temperature=0.1)

predict_messages

위에서는 텍스트를 predict 하는 방법을 살펴 봤다.  
질문을 했더니 답변을 받았지.

이번에는 message 들을 predict 해보자.

우선 message constructor 들을 import 해야한다.

HumanMessage 는 사람이 보내는 것.  
AIMessage 는 AI에 의해 보내지는 것이다.  
SystemMessage 는 우리가 LLM에 설정들을 제공하기 위한 Message 이다.

아래 messages: list 를 작성해보자.


In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. and you only reply in korean",
    ),
    AIMessage(content="안녕, 나는 폴이야."),
    HumanMessage(content="서울과 부산사이의 거리는 얼마인가? 그리고 너의 이름은?"),
]

SystemMessage - 이걸로 우린 AI에 일종의 기본 설정, 기본 값, 기본 context를 설정한 것이다.

AIMessage - string을 미리 만들어두고 가상의 대화를 만들었다. (답변한 것처럼 가정)

HumanMessage - 사용자로서 질문.

단순히 string들을 predict하는 것 대신 predict_messages 를 사용하고 message의 list를 넘겨준다.


In [ ]:
chat.predict_messages(messages)

AIMessage(content='서울과 부산 사이의 거리는 약 325km입니다. 제 이름은 폴이에요.')

string을 predict 하는 것과  
message들의 list인 messages를 predict 하는 것의 차이.

다음으로는 위에서 만든 대화 messages 를 조금 더 커스터마이징 하는 방법에 대해 알아보자.
지금은 message를 우리가 하드코딩하고 있으니까.

하드코딩 대신 일종의 placeholder 를 넣는다면 prompt 를 더 유용하게 만들 수 있다.

- 'korean 대신 {language} '
- '폴 대신 {name}'
- '서울 대신 {city_a}'
- '부산 대신 {city_b}'


In [ ]:
messages = [
    SystemMessage(
        content="You are a geography expert. and you only reply in {language}",
    ),
    AIMessage(content="안녕, 나는 {name}이야."),
    HumanMessage(content="{city_a}과 {city_b}사이의 거리는 얼마인가? 그리고 너의 이름은?"),
]

Prompt Templates

prompt 는 llm과 의사소통을 할 수 있는 유일한 방법이기 때문에 매우 중요하다.  
그렇기 떄문에 prompt의 성능이 좋다면 LLM의 답변도 좋아질 것이다.

매우 중요!! Prompt!!!!

prompt끼리 결합도 할 수있고, 저장하거나 불러올 수도 있다.

chat prompt template 이전에, 일반 prompt template 을 먼저 살펴보자.

normal prompt template 는 text를 predict할 때 사용한다.


In [ ]:
# normal prompt template - 그냥 string을 이용해서 template를 만든다.
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("{city_a}과 {city_b}사이의 거리는 얼마인가?")

# format으로 유효성 검사 - 만약 placeholder로 지정한 곳에 값을 넣지 않으면 에러 발생.
template.format(city_a="서울", city_b="부산")

'서울과 부산사이의 거리는 얼마인가?'

template.format()을 실행하면 string이 return 된다.   

template를 확인했으면 prompt 변수에 넣어준다.

In [ ]:
prompt = template.format(city_a="서울", city_b="부산")

먼저 template를 만들었고, format 메서드의 호출 결과를 prompt에 저장.

prompt가 있다면 이제 predict 해보자.

In [ ]:
chat.predict(prompt)

'서울과 부산 사이의 거리는 약 325km 입니다.'

여기까지 작성하는 과정에서 template.format과 predict를 반복해서 작성하는 게 귀찮다고 느낄 수 있다.  
이 후 langchain expression language 라는 것의 사용으로 template를 format 한 후 chat model 에게 전달하고 predict 를 호출하는 과정까지  
한 줄의 코드로 구현하게 될 것이다. 좀만 기둘..

전체코드 살펴보자.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# chat model: 대화에 특화된 모델
chat = ChatOpenAI(
    temperature=0.1
)

template = PromptTemplate.from_template(
    "What is the distance between {city_a} and {city_b}."
)

prompt = template.format(city_a="부평", city_b="송도")

chat.predict(prompt)

'The distance between 부평 (Bupyeong) and 송도 (Songdo) in South Korea is approximately 10 kilometers (6.2 miles) by road.'

ChatPromptTemplate 로 위 코드를 개선해보자.

ChatPromptTemplate를 import 하면 from langchain.schema import HumanMessage, AIMessage, SystemMessage 를 작성하지 않아도 좋다.  
ChatPromptTemplate.from_messages() 를 사용한다.

.from_messages 에는 똑같이 message_list를 전달한다.


In [ ]:
# Chat prompt template - template를 messages 로부터 만든다.
from langchain.prompts import ChatPromptTemplate

# messages = [
#     SystemMessage(
#         content="You are a geography expert. and you only reply in {language}",
#     ),
#     AIMessage(content="안녕, 나는 {name}이야."),
#     HumanMessage(content="{city_a}과 {city_b}사이의 거리는 얼마인가? 그리고 너의 이름은?"),
# ]


# messages 는 from_messages 의 내용으로 들어간다.
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. and you only reply in {language}"),
    ("ai", "안녕, 나는 {name}이야."),
    ("human", "{city_a}과 {city_b}사이의 거리는 얼마인가? 그리고 너의 이름은?"),
])


message를 format하기 위해서 template.format_message() 를 실행해 보자.

In [ ]:
template.format_messages(
    language="korean",
    name="폴",
    city_a="부평",
    city_b="송도",
)

[SystemMessage(content='You are a geography expert. and you only reply in korean'),
 AIMessage(content='안녕, 나는 폴이야.'),
 HumanMessage(content='부평과 송도사이의 거리는 얼마인가? 그리고 너의 이름은?')]

template 를 확인했으면 prompt 에 넣어준다.  
그리고 LLM에게 보내보자.  

In [ ]:
prompt = template.format_messages(
    language="korean",
    name="폴",
    city_a="부평",
    city_b="송도",
)

chat.predict_messages(prompt)

AIMessage(content='부평과 송도 사이의 거리는 약 20km 정도입니다. 제 이름은 폴이에요.')

다음(langchain expression language)으로 나아가기 전에 output parser에 대해서 조금 알아보자.  

Output Parser 는 LLM의 output을 구조별로 parse(파싱) 할 수 있게 해준다.

langchain expression language 는 우리의 코드를 굉장히 줄여준다.  
그리고 다양한 template 와 LLM 호출, 서로 다른 응답(response)을 함께 사용하게 해준다.  


일단 먼저, Output Parser 를 만드는 것부터 해보자.

output parser가 필요한 이유는, LLM의 응답(response)을 변형해야 할 때가 있기 때문이다.  

이번에는 응답을 list 로 변환시키는 것을 만들어보자.  


종종 LLM의 응답을 변형시키고 싶을때가 생길것이다.  
왜냐하면 LLM은 항상 텍스트로만 응답하니까 그 응답을 무언가로 변형(transform)시키고 싶을 거다.  
데이터베이스에 넣을 형태라던지, dictionary 또는 tuple에 저장할 수 있도록 말이다.  

이것이 바로 output parser 가 필요한 이유이다.

먼저 BaseOutputParser 를 불러와서 사용해보자.  
BaseOutputParser 는 parse 라는 메서드를 꼭 구현해야 한다.


In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [ ]:
template = ChatPromptTemplate.from_messages([
    # 최대 max_items 개 만큼 리스트를 만들어줘라, list가 아닌 것으로는 답을 하지 말아라.
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in uppercase. Do not reply with anything elss."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

chat.predict_messages(prompt)


AIMessage(content='MERCURY, VENUS, EARTH, MARS, JUPITER, SATURN, URANUS, NEPTUNE')

Outputparser 사용해보자.  

predict한 결과값을 result에 담아서 parse 해보자.  

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        # 최대 max_items 개 만큼 리스트를 만들어줘라, list가 아닌 것으로는 답을 하지 말아라.
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in uppercase. Do not reply with anything elss."),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(max_items=10, question="What are the planets?")

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['MERCURY',
 'VENUS',
 'EARTH',
 'MARS',
 'JUPITER',
 'SATURN',
 'URANUS',
 'NEPTUNE',
 'PLUTO']

우리는 LLM으로부터 성공적으로 응답을 받았다.  

prompt template를 사용했고  
prompt template을 포맷팅 한 후에 output parse를 사용했다.

우리가 한 작업들을 보면 코드가 너무 많다.  
message format, predict 호출, 이후 parser 만들고 parse method 까지.  
우리는 더 나은 방법이 필요하다. 그리고 실제로 더 좋은 방법이 있다.

Langchain expression language 표현언어에 대해서 알아보자.  
우리에게 필요한 건 template, output parser 그리고 chat model 이 전부이다.  
우리가 만들 건 <b>"Chain"</b> 이다.  

'Chain'이라는 것은 기본적으로 모든 요소를 합쳐주는 역할을 한다.  
합쳐진 요소들은 하나의 chain으로 실행될 것이고, 하나하나 순서대로 result를 반환할 때 까지 



In [ ]:
# 랭체인이 | 기호에 마법을 부여해준다.
chain = template | chat | CommaOutputParser()

위 Chain은 template, language model, 그리고 output parser로 구성되어 있다.  
chain을 작동시켜보자.  
chain.invoke() - invoke 메서드의 입력값으로는 dictionary 타입이 들어가야 한다.

In [ ]:
# placeholder를 채워줘라
chain.invoke(
    {
        "max_items": "5",
        "question": "What are the pokemons?",
    }
)

['PIKACHU', 'BULBASAUR', 'CHARIZARD', 'SQUIRTLE', 'JIGGLYPUFF']

전체 코드를 보자.

In [ ]:
from langchain.schema import BaseOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# chat model: 대화에 특화된 모델
chat = ChatOpenAI(
    temperature=0.1
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

template = ChatPromptTemplate.from_messages(
    [
        # 최대 max_items 개 만큼 리스트를 만들어줘라, list가 아닌 것으로는 답을 하지 말아라.
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in uppercase. Do not reply with anything elss."),
        ("human", "{question}"),
    ]
)

# 랭체인이 | 기호에 마법을 부여해준다.
chain = template | chat | CommaOutputParser()

# placeholder를 채워줘라
chain.invoke(
    {
        "max_items": "5",
        "question": "What are the pokemons?",
    }
)

['PIKACHU', 'BULBASAUR', 'CHARIZARD', 'SQUIRTLE', 'EEVEE']

코드가 많이 단축된게 보일 것이다.

물론 PromptTemplate, OutputParser, chat model 코드가 있긴 하다.  
하지만 우리가 한 것은 모든걸 한줄로 늘어놓은게 다 이다.  
랭체인 내부에서는 우리를 위해 .format_messages()를 호출하고, 그 다음에 chat.predict 도 호출하고, 마지막으로 parse를 호출해 준 것이다.   
이 모든건 invoke() 호출 한방이다.

이 문법과 함께한다면 더 멋진 일들을 많이 할 수 있다.  
왜냐하면 이걸로 chain 들 끼리도 결합을 할 수 있기 때문이다.  

예를 들어)
chain_one = template | chat | CommaOutputParser()  
chain_two = template_2 | chat2 | OutputParser()  

all = chain_one | chain_two | output3

먼저 chain_one 을 넣고 그 출력값을 chain_two 의 입력값으로 사용할 수 있다.  
거기에 또 다른 output parser도 놓을 수 있지(output3)



LangChain expression language 표현언어에 대해 조금 더 살펴보자.  

먼저 구성요소(component)부터 살펴보자.  
chain이 가질 수 있는 구성요소는 다음과 같다.  
- prompt
- retriever
- llm, chatmodel
- tool
- outputparser

우리는 이 전에 Prompt, Chatmodel, OutputParser 를 사용했다.  
prompt 의 입력타입은 Dictionary, 출력타입은 PromptValue,  
chatmodel의 입력타입은 single string, list of chat messages or a PromptValue, 출력타입은 ChatMessage,  
ouputParser의 입력타입은 ChatMessage or LLM . 

이제 무슨일이 일어나는지 이해할 수 있을 것이다.  

In [ ]:
chain = template | chat | CommaOutputParser()

chain.invoke(
    {
        "max_items": "5",
        "question": "What are the pokemons?",
    }
)

처음에 
```python
chain.invoke(
    {
        "max_items": "5",
        "question": "What are the pokemons?",
    }
)
```
dictionary 값은 template 에 주어지고, template의 반환값은 chat으로 전달된다.  
그리고 chat의 반환값은 OutputParser에게 전달된다.  

chain은 단지 component의 그룹 혹은 나열이라고 보면 될 것이다.  
각 구성요소는 입력값을 받아서 출력값을 반환한다.  
각 출려값은 chain에 입력한 코드가 끝날 때까지 옆의 구성요소에게 전달되고 사용자는 최종 출력값을 전달받는 것이다.


두 개의 chain끼리 연결하는 걸 해보자.  



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

# 요리사
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 세계 레벨의 요리사야. 너는 어떤 종류의 요리든 쉽게 구할 수 있는 재료로 따라하기 쉬운 레시피를 만들어낼 수 있어."),
    ("human", "나는 {cuisine}를 요리하고 싶어요."),
])

chef_chain = chef_prompt | chat


In [ ]:
# 비건 요리사
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 채식주의자를 위한 요리사야. 전통적인 채식주의자용 레시피에 특화되어 있어. 너는 대체 재료를 찾고, 준비하는 방법에 대해 설명해줄수있어. 대신, 기존 레시피를 너무 많이 변경해서는 안돼. 만약 대체품이 없다면 그냥 모른다고 말해줘."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat


마지막 chain 을 만들어보자.  필요한 입력값들이 뭔지 생각해보자.

일단 recipe 가 하나의 입력값이 될 것이다.  
그리고 그 다음은 cuisine.

chef_chain 이 첫 번째 chain이 될 것이다.  
왜냐하면 먼저 레시피 값을 얻고, 그 다음 채식주의자 레시피로 변경해야 하니까.

In [ ]:
# final_chain = chef_chain | veg_chain
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "라면"})

AIMessage(content='이 레시피는 매우 간단하고 맛있어 보이네요! 채식주의자를 위해 대체할 수 있는 재료가 있을까요? 계란은 채식주의 식단에 포함되지 않는데, 대체할 수 있는 재료로는 토푸, 두부, 또는 식물성 달걀 대체품이 있습니다. 계란 대신 토푸를 사용하면 비슷한 식감을 낼 수 있을 것 같아요. 토푸를 물에 삶아서 계란 대신 사용하면 될 것 같습니다. 계란 대신 토푸를 사용하여 라면을 만들어보세요!')

잘 동작하니 전체 코드로 살펴보자.
+ streaming

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 요리사
chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 세계 레벨의 요리사야. 너는 어떤 종류의 요리든 쉽게 구할 수 있는 재료로 따라하기 쉬운 레시피를 만들어낼 수 있어."),
        ("human", "나는 {cuisine}를 요리하고 싶어요."),
    ]
)

chef_chain = chef_prompt | chat

# 비건 요리사
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 채식주의자를 위한 요리사야. 전통적인 채식주의자용 레시피에 특화되어 있어. 너는 대체 재료를 찾고, 준비하는 방법에 대해 설명해줄수있어. 대신, 기존 레시피를 너무 많이 변경해서는 안돼. 만약 대체품이 없다면 그냥 모른다고 말해줘.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

# 최종 chain
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "라면"})

좋아요! 라면은 매우 쉽게 만들 수 있는 요리입니다. 다음은 간단한 라면 레시피입니다.

재료:
- 라면 봉지 1개
- 물 2컵
- 계란 1개
- 녹말가루 1큰술
- 소금 약간
- 후추 약간
- 김가루 약간
- 파 또는 대파 약간

1. 물을 냄비에 넣고 끓입니다.
2. 물이 끓기 시작하면 라면 봉지를 넣고 3분간 삶아줍니다.
3. 계란을 껍질을 벗겨 볼에 깨뜨려 넣고 녹말가루와 소금을 넣어 풀어줍니다.
4. 라면에 계란물을 넣고 잘 섞어줍니다.
5. 후추를 약간 넣고 김가루와 파를 올려 완성합니다.

간단하고 맛있는 라면 요리 완성! 맛있게 드세요.이 라면 레시피는 매우 간단하고 맛있어 보이네요! 만약 채식주의자를 위한 대체 재료를 사용하고 싶다면, 계란 대신 토푸를 사용할 수 있습니다. 토푸는 대체로 계란과 비슷한 맛과 질감을 제공해주는 대체품이에요. 

대체로 계란 대신 토푸를 사용할 때는, 토푸를 깨끗이 씻어서 물기를 제거하고 녹말가루와 소금을 섞어 계란물을 만드는 단계에서 토푸를 사용하면 됩니다. 그 외의 과정은 동일하게 따라가면 됩니다. 

채식주의자용 라면 레시피로서 토푸를 사용하면 계란 대신 대체재료를 활용하여 맛있는 채식주의자용 라면을 즐길 수 있을 거에요.

AIMessageChunk(content='이 라면 레시피는 매우 간단하고 맛있어 보이네요! 만약 채식주의자를 위한 대체 재료를 사용하고 싶다면, 계란 대신 토푸를 사용할 수 있습니다. 토푸는 대체로 계란과 비슷한 맛과 질감을 제공해주는 대체품이에요. \n\n대체로 계란 대신 토푸를 사용할 때는, 토푸를 깨끗이 씻어서 물기를 제거하고 녹말가루와 소금을 섞어 계란물을 만드는 단계에서 토푸를 사용하면 됩니다. 그 외의 과정은 동일하게 따라가면 됩니다. \n\n채식주의자용 라면 레시피로서 토푸를 사용하면 계란 대신 대체재료를 활용하여 맛있는 채식주의자용 라면을 즐길 수 있을 거에요.')

위에서 최종 chain 을 만든 부분을 보면 langchaing 언어로 되어 있는데
```python
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "라면"})
```
이 코드는 langchain이 코드({"recipe": chef_chain})를 먼저 실행하고 응답 결과를 다음(veg_chain)으로 전달하도록 한다.  
이 걸 RunnableMap 이라고 한다.  앞에 것이 먼저 실행되고, 그 출력 값이 chain의 다음 요소에게 전달되는 것!
이렇게 하는 이유는 veg_chain에 recipe 값이 필요하기 때문이다.  

veg_chain이 호출되면 처음에는 prompt가 호출되고, prompt의 결과값은 chat에게 전달된다.
prompt는 필요로 하는 값을 전달받아야 하는데, 여기서는 recipe이 그것이다.  




